# Performance comparaison

| Fingerprint | R2 | RMSE |  
|:-:|:-:|:-:|  
| ECFP| 0.765 | 0.9808 |
|Can2Can|0.7176|1.073|
|Enum2Enum|0.725|1.059|
|Transformer|0.862|0.750|
| NFP| 0.8845 | 0.6868 |

In [11]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

import chainer
from chainer import serializers
from seq2seq.seq2seq import Seq2seq, load_vocabulary
import sys
sys.path.append('./transformer')
from net import Transformer
import preprocess
import optuna

UNK, EOS = 0, 1

In [3]:
df_train = pd.read_csv('data/sol_train.csv')
df_test = pd.read_csv('data/sol_test.csv')
df_train.head()

,SMILES,unknown,solubility,processed_smiles,spaced
0,[nH0]1c(SC)c2c([nH0]cc[nH0]2)[nH0]c1,6966-78-5,-2.36,[ n H 0 ] 1 c ( S C ) c 2 c ( [ n H 0 ] c c [ ...,[ n H 0 ] 1 c ( S C ) c 2 c ( [ n H 0 ] c c [ ...
1,CCC(C)Cl,78-86-4,-1.96,C C C ( C ) Cl,C C C ( C ) C l
2,O=C(NC(=O)c1ccccc1)c1ccccc1,614-28-8,-2.27,O = C ( N C ( = O ) c 1 c c c c c 1 ) c 1 c c ...,O = C ( N C ( = O ) c 1 c c c c c 1 ) c 1 c c ...
3,CC(C(C)(C)C)O,464-07-3,-0.62,C C ( C ( C ) ( C ) C ) O,C C ( C ( C ) ( C ) C ) O
4,[O-][N+](c1c(O)cccc1)=O,88-75-5,-1.74,[ O- ] [ N+ ] ( c 1 c ( O ) c c c c 1 ) = O,[ O - ] [ N + ] ( c 1 c ( O ) c c c c 1 ) = O


In [4]:
x_train = df_train['processed_smiles']
y_train = df_train['solubility']
x_test = df_test['processed_smiles']
y_test = df_test['solubility']

In [5]:
def load_data(vocabulary, lst):
    data = []
    for l in lst:
        words = l.strip().split()
        array = np.array([vocabulary.get(w, UNK) for w in words], np.int32)
        data.append(array)
    return data

In [6]:
source_ids = load_vocabulary('data/Enum2Enum/vocab2.txt')
xnum_train = load_data(source_ids, x_train)
xnum_test = load_data(source_ids, x_test)

# Encode to fingerprint

In [7]:
# Can2Can
model = Seq2seq(1, 46, 46, 256)
chainer.serializers.load_npz('../result/can2can_iter_132000.npz', model)

In [31]:
# Enum2Enum
model = Seq2seq(2, 43, 43, 256)
chainer.serializers.load_npz('../result/Enum2Enum/model_epoch_3.npz', model)

In [8]:
X_train = model.encode(xnum_train)[0].data
X_test = model.encode(xnum_test)[0].data

In [25]:
# Transformer
x_train = df_train['spaced']
y_train = df_train['solubility']
x_test = df_test['spaced']
y_test = df_test['solubility']

In [27]:
en_path = os.path.join('data/Enum2Enum', 'sval.txt')
source_vocab = ['<eos>', '<unk>', '<bos>'] + \
    preprocess.count_words(en_path, 50)
source_ids = {word: index for index, word in enumerate(source_vocab)}
source_words = {i: w for w, i in source_ids.items()}

100% (500000 of 500000) |################| Elapsed Time: 0:00:41 Time:  0:00:41


In [29]:
# Transformer
model = Transformer(2, 38, 38,
        256,
        h=4,
        dropout=0.1,
        max_length=500,
        use_label_smoothing=False,
        embed_position=False)

def encode(x):
    words = preprocess.split_sentence(x)
    x = model.xp.array([source_ids.get(w, 1) for w in words], 'i')
    h = model.encode([x])
    return np.mean(h.data[0], axis=1)

chainer.serializers.load_npz('transformer/result/best_model.npz', model)

In [30]:
X_train = [encode(x_train[i]) for i in range(len(x_train))]
X_test = [encode(x_test[i]) for i in range(len(x_test))]

# Prediction
## Can2Can
### MLP

In [14]:
# Default
n = 10
r2 = np.zeros(n)
mse = np.zeros(n)

for i in range(n):
    MLP = MLPRegressor()
    MLP.fit(X_train, y_train)
    y_pred = MLP.predict(X_test)
    r2[i] = r2_score(y_test, y_pred)
    mse[i] = mean_squared_error(y_test, y_pred)

print("Test R2: {:.4f} ± {:.4f}".format(np.mean(r2), np.std(r2)))
print("Test MSE: {:.4f} ± {:.4f}".format(np.mean(mse), np.std(mse)))

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

Test R2: 0.6502 ± 0.0132
Test MSE: 1.4281 ± 0.0537


In [12]:
def objective_mlp(trial):
    n_layers = trial.suggest_int('n_layers', 1,3)
    layers = []
    for i in range(n_layers):
        n_units = int(trial.suggest_loguniform('n_units_l{}'.format(i), 1, 1000))
        layers.append(n_units)
    
    n_folds = 4
    score = 0
    for _ in range(n_folds):
        mlp = MLPRegressor(hidden_layer_sizes=layers)
        X_trn, X_val, y_trn, y_val = train_test_split(X_train, y_train)
        mlp.fit(X_trn, y_trn)
        y_pred = mlp.predict(X_val)
        score += mean_squared_error(y_val, y_pred)
    return score/n_folds

study = optuna.create_study()
study.optimize(objective_mlp, n_trials=100)

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

[I 2019-03-27 17:31:12,551] Finished a trial resulted in value: 1.8102511558439862. Current best value is 1.5099048867093632 with parameters: {'n_layers': 3, 'n_units_l0': 601.7334661991075, 'n_units_l1': 34.53736314637694, 'n_units_l2': 2.755145401102368}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
 

[I 2019-03-27 17:34:02,716] Finished a trial resulted in value: 1.6319562296217538. Current best value is 1.5099048867093632 with parameters: {'n_layers': 3, 'n_units_l0': 601.7334661991075, 'n_units_l1': 34.53736314637694, 'n_units_l2': 2.755145401102368}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
 

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 17:34:27,496] Finished a trial resulted in value: 1.5847131440331066. Current best value is 1.4357352659177811 with parameters: {'n_layers': 1, 'n_units_l0': 5.690829667369596}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/py

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 17:35:17,497] Finished a trial resulted in value: 1.6765618101326634. Current best value is 1.4357352659177811 with parameters: {'n_layers': 1, 'n_units_l0': 5.690829667369596}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/py

...

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 17:39:31,708] Finished a trial resulted in value: 1.535399939095107. Current best value is 1.415874610074891 with parameters: {'n_layers': 1, 'n_units_l0': 38.93661768724165}.
/home/honda/anaconda3/lib/pyth

[I 2019-03-27 17:39:58,721] Finished a trial resulted in value: 1.5207723947503418. Current best value is 1.415874610074891 with parameters: {'n_layers': 1, 'n_units_l0': 38.93661768724165}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 17:40:18,217] Finished a trial resulted in value: 1.538391403618202. Current best value is 1.415874610074891 with parameters: {'n_layers': 1, 'n_units_l0': 38.93661768724165}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:5

[I 2019-03-27 17:40:46,902] Finished a trial resulted in value: 1.7357177859804827. Current best value is 1.415874610074891 with parameters: {'n_layers': 1, 'n_units_l0': 38.93661768724165}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/pyt

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 17:41:23,256] Finished a trial resulted in value: 2.5464283984946734. Current best value is 1.415874610074891 with parameters: {'n_layers': 1, 'n_units_l0': 38.93661768724165}.
[I 2019-03-27 17:41:38,666] Finished a trial resulted in value: 1.5587180119236945. Current best value is 1.415874610074891 with parameters: {'n_layers': 1, 'n_units_l0': 38.93661768724165}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:

[I 2019-03-27 17:42:41,716] Finished a trial resulted in value: 1.5282648959110248. Current best value is 1.415874610074891 with parameters: {'n_layers': 1, 'n_units_l0': 38.93661768724165}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/pyt

In [15]:
# Optimized

n = 10
r2 = np.zeros(n)
mse = np.zeros(n)

for i in range(n):
    MLP = MLPRegressor((39))
    MLP.fit(X_train, y_train)
    y_pred = MLP.predict(X_test)
    r2[i] = r2_score(y_test, y_pred)
    mse[i] = mean_squared_error(y_test, y_pred)

print("Test R2: {:.4f} ± {:.4f}".format(np.mean(r2), np.std(r2)))
print("Test MSE: {:.4f} ± {:.4f}".format(np.mean(mse), np.std(mse)))

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

Test R2: 0.6423 ± 0.0121
Test MSE: 1.4605 ± 0.0493


### RF

In [16]:
#Default
n = 10
r2 = np.zeros(n)
mse = np.zeros(n)

for i in range(n):
    RF = RandomForestRegressor()
    RF.fit(X_train, y_train)
    y_pred = RF.predict(X_test)
    r2[i] = r2_score(y_test, y_pred)
    mse[i] = mean_squared_error(y_test, y_pred)

print("Test R2: {:.4f} ± {:.4f}".format(np.mean(r2), np.std(r2)))
print("Test MSE: {:.4f} ± {:.4f}".format(np.mean(mse), np.std(mse)))

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/honda/anaconda3/lib/python3.6/site-package

Test R2: 0.6287 ± 0.0243
Test MSE: 1.5159 ± 0.0993


In [17]:
def objective_rf(trial):
    max_depth = int(trial.suggest_loguniform('max_depth', 2, 100))
    n_estimators = int(trial.suggest_loguniform('n_estimators', 2, 1000))
    max_features = trial.suggest_int('max_features', 1, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        
    n_folds = 4
    score = 0
    for _ in range(n_folds):
        rf = RandomForestRegressor(max_depth=max_depth, n_estimators=n_estimators, max_features=max_features,
                              min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
        X_trn, X_val, y_trn, y_val = train_test_split(X_train, y_train)
        rf.fit(X_trn, y_trn)
        y_pred = rf.predict(X_val)
        score += mean_squared_error(y_val, y_pred)
    return score/n_folds

study = optuna.create_study()
study.optimize(objective_rf, n_trials=100)

[I 2019-03-27 17:47:02,562] Finished a trial resulted in value: 2.291075054629868. Current best value is 2.291075054629868 with parameters: {'max_depth': 19.97957442135277, 'n_estimators': 14.683033705451166, 'max_features': 1, 'min_samples_split': 2, 'min_samples_leaf': 6}.
[I 2019-03-27 17:47:02,650] Finished a trial resulted in value: 2.2376994718325247. Current best value is 2.2376994718325247 with parameters: {'max_depth': 22.019384922918128, 'n_estimators': 4.3585164019968206, 'max_features': 7, 'min_samples_split': 4, 'min_samples_leaf': 5}.
[I 2019-03-27 17:47:06,114] Finished a trial resulted in value: 2.643137045694201. Current best value is 2.2376994718325247 with parameters: {'max_depth': 22.019384922918128, 'n_estimators': 4.3585164019968206, 'max_features': 7, 'min_samples_split': 4, 'min_samples_leaf': 5}.
[I 2019-03-27 17:47:07,666] Finished a trial resulted in value: 1.9739687343666112. Current best value is 1.9739687343666112 with parameters: {'max_depth': 32.86182035

[I 2019-03-27 17:48:12,437] Finished a trial resulted in value: 2.7183151906580694. Current best value is 1.6563074203771297 with parameters: {'max_depth': 14.344749683916975, 'n_estimators': 106.20375195835145, 'max_features': 6, 'min_samples_split': 8, 'min_samples_leaf': 6}.
[I 2019-03-27 17:48:15,586] Finished a trial resulted in value: 2.1791123752695096. Current best value is 1.6563074203771297 with parameters: {'max_depth': 14.344749683916975, 'n_estimators': 106.20375195835145, 'max_features': 6, 'min_samples_split': 8, 'min_samples_leaf': 6}.
[I 2019-03-27 17:48:15,862] Finished a trial resulted in value: 3.2005348064715067. Current best value is 1.6563074203771297 with parameters: {'max_depth': 14.344749683916975, 'n_estimators': 106.20375195835145, 'max_features': 6, 'min_samples_split': 8, 'min_samples_leaf': 6}.
[I 2019-03-27 17:48:15,940] Finished a trial resulted in value: 2.5277879087657174. Current best value is 1.6563074203771297 with parameters: {'max_depth': 14.3447

[I 2019-03-27 17:50:22,924] Finished a trial resulted in value: 1.7610484316406896. Current best value is 1.5685099866936014 with parameters: {'max_depth': 10.462927771362471, 'n_estimators': 812.028696277209, 'max_features': 9, 'min_samples_split': 7, 'min_samples_leaf': 1}.
[I 2019-03-27 17:50:30,920] Finished a trial resulted in value: 2.4222210963846913. Current best value is 1.5685099866936014 with parameters: {'max_depth': 10.462927771362471, 'n_estimators': 812.028696277209, 'max_features': 9, 'min_samples_split': 7, 'min_samples_leaf': 1}.
[I 2019-03-27 17:50:31,063] Finished a trial resulted in value: 1.9124055726646882. Current best value is 1.5685099866936014 with parameters: {'max_depth': 10.462927771362471, 'n_estimators': 812.028696277209, 'max_features': 9, 'min_samples_split': 7, 'min_samples_leaf': 1}.
[I 2019-03-27 17:50:31,204] Finished a trial resulted in value: 2.405548792858847. Current best value is 1.5685099866936014 with parameters: {'max_depth': 10.46292777136

[I 2019-03-27 17:52:41,121] Finished a trial resulted in value: 2.265776879110939. Current best value is 1.5685099866936014 with parameters: {'max_depth': 10.462927771362471, 'n_estimators': 812.028696277209, 'max_features': 9, 'min_samples_split': 7, 'min_samples_leaf': 1}.
[I 2019-03-27 17:52:42,095] Finished a trial resulted in value: 1.7760699754747395. Current best value is 1.5685099866936014 with parameters: {'max_depth': 10.462927771362471, 'n_estimators': 812.028696277209, 'max_features': 9, 'min_samples_split': 7, 'min_samples_leaf': 1}.
[I 2019-03-27 17:52:42,303] Finished a trial resulted in value: 1.895282103022127. Current best value is 1.5685099866936014 with parameters: {'max_depth': 10.462927771362471, 'n_estimators': 812.028696277209, 'max_features': 9, 'min_samples_split': 7, 'min_samples_leaf': 1}.
[I 2019-03-27 17:52:42,693] Finished a trial resulted in value: 1.814250944667489. Current best value is 1.5685099866936014 with parameters: {'max_depth': 10.4629277713624

In [18]:
# Optimized

n = 10
r2 = np.zeros(n)
mse = np.zeros(n)

for i in range(n):
    RF = RandomForestRegressor(max_depth=10, n_estimators=812, max_features=9,
                              min_samples_split=7, min_samples_leaf=1)
    RF.fit(X_train, y_train)
    y_pred = RF.predict(X_test)
    r2[i] = r2_score(y_test, y_pred)
    mse[i] = mean_squared_error(y_test, y_pred)

print("Test R2: {:.4f} ± {:.4f}".format(np.mean(r2), np.std(r2)))
print("Test MSE: {:.4f} ± {:.4f}".format(np.mean(mse), np.std(mse)))

Test R2: 0.6179 ± 0.0018
Test MSE: 1.5599 ± 0.0072


## Enum2Enum
### MLP

In [19]:
# Default
n = 10
r2 = np.zeros(n)
mse = np.zeros(n)

for i in range(n):
    MLP = MLPRegressor()
    MLP.fit(X_train, y_train)
    y_pred = MLP.predict(X_test)
    r2[i] = r2_score(y_test, y_pred)
    mse[i] = mean_squared_error(y_test, y_pred)

print("Test R2: {:.4f} ± {:.4f}".format(np.mean(r2), np.std(r2)))
print("Test MSE: {:.4f} ± {:.4f}".format(np.mean(mse), np.std(mse)))

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

Test R2: 0.6571 ± 0.0106
Test MSE: 1.3999 ± 0.0432


In [20]:
study = optuna.create_study()
study.optimize(objective_mlp, n_trials=100)

[I 2019-03-27 17:59:52,100] Finished a trial resulted in value: 1.6810123726719572. Current best value is 1.6810123726719572 with parameters: {'n_layers': 2, 'n_units_l0': 769.4995119769485, 'n_units_l1': 64.54746270299873}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 18:00:21,987] Finished a trial resulted in value: 4.31013467818729. Current best value is 1.6810123726719572 with parameters: {'n_layers': 2, 'n_units_l0': 769.4995119769485, 'n_units_l1': 64.54746270299873}.
/home/honda/anaconda3/lib/python

[I 2019-03-27 18:00:58,000] Finished a trial resulted in value: 1.4865970692859518. Current best value is 1.4865970692859518 with parameters: {'n_layers': 1, 'n_units_l0': 54.670902155151936}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/p

[I 2019-03-27 18:02:09,184] Finished a trial resulted in value: 1.7040077306911374. Current best value is 1.4865970692859518 with parameters: {'n_layers': 1, 'n_units_l0': 54.670902155151936}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/p

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 18:02:47,384] Finished a trial resulted in value: 2.815295601944131. Current best value is 1.4865970692859518 with parameters: {'n_layers': 1, 'n_units_l0': 54.670902155151936}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/py

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 18:03:18,097] Finished a trial resulted in value: 1.5968026397118489. Current best value is 1.4865970692859518 with parameters: {'n_layers': 1, 'n_units_l0': 54.670902155151936}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/p

...

[I 2019-03-27 18:05:10,154] Finished a trial resulted in value: 1.6556895934860776. Current best value is 1.4865970692859518 with parameters: {'n_layers': 1, 'n_units_l0': 54.670902155151936}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/p

[I 2019-03-27 18:07:06,250] Finished a trial resulted in value: 2.5415821986401683. Current best value is 1.4486623446264901 with parameters: {'n_layers': 3, 'n_units_l0': 696.337032987555, 'n_units_l1': 5.9532576568375335, 'n_units_l2': 96.79537060685016}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
 

[I 2019-03-27 18:16:22,020] Finished a trial resulted in value: 1.624245654954182. Current best value is 1.44599479163038 with parameters: {'n_layers': 2, 'n_units_l0': 875.2499546465023, 'n_units_l1': 88.13995328010041}.
[I 2019-03-27 18:16:28,986] Finished a trial resulted in value: 3.6617890289726045. Current best value is 1.44599479163038 with parameters: {'n_layers': 2, 'n_units_l0': 875.2499546465023, 'n_units_l1': 88.13995328010041}.
[I 2019-03-27 18:17:09,725] Finished a trial resulted in value: 1.6225817446981483. Current best value is 1.44599479163038 with parameters: {'n_layers': 2, 'n_units_l0': 875.2499546465023, 'n_units_l1': 88.13995328010041}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network

[I 2019-03-27 18:20:53,870] Finished a trial resulted in value: 1.6114832633247578. Current best value is 1.4156719169443424 with parameters: {'n_layers': 2, 'n_units_l0': 629.0682900289332, 'n_units_l1': 10.853916801628936}.
[I 2019-03-27 18:21:18,237] Finished a trial resulted in value: 1.429351572755647. Current best value is 1.4156719169443424 with parameters: {'n_layers': 2, 'n_units_l0': 629.0682900289332, 'n_units_l1': 10.853916801628936}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 18:21:26,881] F

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 18:23:25,157] Finished a trial resulted in value: 1.9340959149246952. Current best value is 1.4156719169443424 with parameters: {'n_layers': 2, 'n_units_l0': 629.0682900289332, 'n_units_l1': 10.853916801628936}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWar

In [21]:
# Optimized

n = 10
r2 = np.zeros(n)
mse = np.zeros(n)

for i in range(n):
    MLP = MLPRegressor((629, 11))
    MLP.fit(X_train, y_train)
    y_pred = MLP.predict(X_test)
    r2[i] = r2_score(y_test, y_pred)
    mse[i] = mean_squared_error(y_test, y_pred)

print("Test R2: {:.4f} ± {:.4f}".format(np.mean(r2), np.std(r2)))
print("Test MSE: {:.4f} ± {:.4f}".format(np.mean(mse), np.std(mse)))

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Test R2: 0.6758 ± 0.0057
Test MSE: 1.3236 ± 0.0232


### RF

In [22]:
#Default
n = 10
r2 = np.zeros(n)
mse = np.zeros(n)

for i in range(n):
    RF = RandomForestRegressor()
    RF.fit(X_train, y_train)
    y_pred = RF.predict(X_test)
    r2[i] = r2_score(y_test, y_pred)
    mse[i] = mean_squared_error(y_test, y_pred)

print("Test R2: {:.4f} ± {:.4f}".format(np.mean(r2), np.std(r2)))
print("Test MSE: {:.4f} ± {:.4f}".format(np.mean(mse), np.std(mse)))

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/honda/anaconda3/lib/python3.6/site-package

Test R2: 0.6387 ± 0.0106
Test MSE: 1.4750 ± 0.0434


In [23]:
study = optuna.create_study()
study.optimize(objective_rf, n_trials=100)

[I 2019-03-27 18:41:00,200] Finished a trial resulted in value: 2.3814723656587695. Current best value is 2.3814723656587695 with parameters: {'max_depth': 3.1454442590298304, 'n_estimators': 866.8221259527813, 'max_features': 7, 'min_samples_split': 5, 'min_samples_leaf': 2}.
[I 2019-03-27 18:41:01,106] Finished a trial resulted in value: 2.844269901513285. Current best value is 2.3814723656587695 with parameters: {'max_depth': 3.1454442590298304, 'n_estimators': 866.8221259527813, 'max_features': 7, 'min_samples_split': 5, 'min_samples_leaf': 2}.
[I 2019-03-27 18:41:01,263] Finished a trial resulted in value: 1.9668347824700754. Current best value is 1.9668347824700754 with parameters: {'max_depth': 31.431617978839412, 'n_estimators': 13.450056743420534, 'max_features': 5, 'min_samples_split': 6, 'min_samples_leaf': 9}.
[I 2019-03-27 18:41:01,450] Finished a trial resulted in value: 2.0100769199009845. Current best value is 1.9668347824700754 with parameters: {'max_depth': 31.4316179

[I 2019-03-27 18:41:38,820] Finished a trial resulted in value: 1.9862033794055995. Current best value is 1.6279030887882195 with parameters: {'max_depth': 40.6009114330541, 'n_estimators': 41.554331820773605, 'max_features': 8, 'min_samples_split': 10, 'min_samples_leaf': 6}.
[I 2019-03-27 18:41:39,549] Finished a trial resulted in value: 2.7235754153140266. Current best value is 1.6279030887882195 with parameters: {'max_depth': 40.6009114330541, 'n_estimators': 41.554331820773605, 'max_features': 8, 'min_samples_split': 10, 'min_samples_leaf': 6}.
[I 2019-03-27 18:41:40,104] Finished a trial resulted in value: 1.6289979122236529. Current best value is 1.6279030887882195 with parameters: {'max_depth': 40.6009114330541, 'n_estimators': 41.554331820773605, 'max_features': 8, 'min_samples_split': 10, 'min_samples_leaf': 6}.
[I 2019-03-27 18:41:40,759] Finished a trial resulted in value: 1.726200489978912. Current best value is 1.6279030887882195 with parameters: {'max_depth': 40.60091143

[I 2019-03-27 18:44:49,226] Finished a trial resulted in value: 1.590600839204951. Current best value is 1.5083468807782512 with parameters: {'max_depth': 34.840493965736485, 'n_estimators': 986.3704877190894, 'max_features': 9, 'min_samples_split': 2, 'min_samples_leaf': 1}.
[I 2019-03-27 18:44:52,967] Finished a trial resulted in value: 1.7811533586940718. Current best value is 1.5083468807782512 with parameters: {'max_depth': 34.840493965736485, 'n_estimators': 986.3704877190894, 'max_features': 9, 'min_samples_split': 2, 'min_samples_leaf': 1}.
[I 2019-03-27 18:44:57,330] Finished a trial resulted in value: 1.9348519840904526. Current best value is 1.5083468807782512 with parameters: {'max_depth': 34.840493965736485, 'n_estimators': 986.3704877190894, 'max_features': 9, 'min_samples_split': 2, 'min_samples_leaf': 1}.
[I 2019-03-27 18:44:59,508] Finished a trial resulted in value: 1.8046495452765534. Current best value is 1.5083468807782512 with parameters: {'max_depth': 34.84049396

[I 2019-03-27 18:46:23,613] Finished a trial resulted in value: 1.643555056655357. Current best value is 1.5083468807782512 with parameters: {'max_depth': 34.840493965736485, 'n_estimators': 986.3704877190894, 'max_features': 9, 'min_samples_split': 2, 'min_samples_leaf': 1}.
[I 2019-03-27 18:46:24,312] Finished a trial resulted in value: 1.6965577350794063. Current best value is 1.5083468807782512 with parameters: {'max_depth': 34.840493965736485, 'n_estimators': 986.3704877190894, 'max_features': 9, 'min_samples_split': 2, 'min_samples_leaf': 1}.
[I 2019-03-27 18:46:24,667] Finished a trial resulted in value: 1.8738607102158833. Current best value is 1.5083468807782512 with parameters: {'max_depth': 34.840493965736485, 'n_estimators': 986.3704877190894, 'max_features': 9, 'min_samples_split': 2, 'min_samples_leaf': 1}.
[I 2019-03-27 18:46:25,454] Finished a trial resulted in value: 1.7744741943436586. Current best value is 1.5083468807782512 with parameters: {'max_depth': 34.84049396

In [24]:
# Optimized

n = 10
r2 = np.zeros(n)
mse = np.zeros(n)

for i in range(n):
    RF = RandomForestRegressor(max_depth=35, n_estimators=986, max_features=9,
                              min_samples_split=2, min_samples_leaf=1)
    RF.fit(X_train, y_train)
    y_pred = RF.predict(X_test)
    r2[i] = r2_score(y_test, y_pred)
    mse[i] = mean_squared_error(y_test, y_pred)

print("Test R2: {:.4f} ± {:.4f}".format(np.mean(r2), np.std(r2)))
print("Test MSE: {:.4f} ± {:.4f}".format(np.mean(mse), np.std(mse)))

Test R2: 0.6402 ± 0.0023
Test MSE: 1.4688 ± 0.0095


## Transformer
### MLP

In [31]:
# Default
n = 10
r2 = np.zeros(n)
mse = np.zeros(n)

for i in range(n):
    MLP = MLPRegressor()
    MLP.fit(X_train, y_train)
    y_pred = MLP.predict(X_test)
    r2[i] = r2_score(y_test, y_pred)
    mse[i] = mean_squared_error(y_test, y_pred)

print("Test R2: {:.4f} ± {:.4f}".format(np.mean(r2), np.std(r2)))
print("Test MSE: {:.4f} ± {:.4f}".format(np.mean(mse), np.std(mse)))

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

Test R2: 0.8464 ± 0.0057
Test MSE: 0.6271 ± 0.0234


In [32]:
study = optuna.create_study()
study.optimize(objective_mlp, n_trials=100)

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

[I 2019-03-27 19:05:07,876] Finished a trial resulted in value: 1.092241616090808. Current best value is 0.649292208312422 with parameters: {'n_layers': 1, 'n_units_l0': 88.76031418639229}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/pyth

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 19:06:05,659] Finished a trial resulted in value: 0.7098399952478203. Current best value is 0.649292208312422 with parameters: {'n_layers': 1, 'n_units_l0': 88.76031418639229}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/pyt

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 19:08:06,647] Finished a trial resulted in value: 0.6910402577279486. Current best value is 0.649292208312422 with parameters: {'n_layers': 1, 'n_units_l0': 88.76031418639229}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/pyt

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 19:10:29,496] Finished a trial resulted in value: 2.4091547088671486. Current best value is 0.649292208312422 with parameters: {'n_layers': 1, 'n_units_l0': 88.76031418639229}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/pyt

...

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 19:15:04,834] Finished a trial resulted in value: 0.7445283100438008. Current best value is 0.6245894499044138 with parameters: {'n_layers': 1, 'n_units_l0': 58.78949075813588}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/py

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 19:15:58,283] Finished a trial resulted in value: 0.7232121755476626. Current best value is 0.6245894499044138 with parameters: {'n_layers': 1, 'n_units_l0': 58.78949075813588}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/py

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 19:17:04,042] Finished a trial resulted in value: 0.7602311092326377. Current best value is 0.6245894499044138 with parameters: {'n_layers': 1, 'n_units_l0': 58.78949075813588}.
/home/honda/anaconda3/lib/py

[I 2019-03-27 19:17:51,706] Finished a trial resulted in value: 0.734368924979903. Current best value is 0.6245894499044138 with parameters: {'n_layers': 1, 'n_units_l0': 58.78949075813588}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/pyt

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
[I 2019-03-27 19:19:28,348] Finished a trial resulted in value: 0.7136375326950193. Current best value is 0.6245894499044138 with parameters: {'n_layers': 1, 'n_units_l0': 58.78949075813588}.
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/py

In [33]:
# Optimized
n = 10
r2 = np.zeros(n)
mse = np.zeros(n)

for i in range(n):
    MLP = MLPRegressor(59)
    MLP.fit(X_train, y_train)
    y_pred = MLP.predict(X_test)
    r2[i] = r2_score(y_test, y_pred)
    mse[i] = mean_squared_error(y_test, y_pred)

print("Test R2: {:.4f} ± {:.4f}".format(np.mean(r2), np.std(r2)))
print("Test MSE: {:.4f} ± {:.4f}".format(np.mean(mse), np.std(mse)))

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.

Test R2: 0.8421 ± 0.0040
Test MSE: 0.6446 ± 0.0161


### RF

In [34]:
#Default
n = 10
r2 = np.zeros(n)
mse = np.zeros(n)

for i in range(n):
    RF = RandomForestRegressor()
    RF.fit(X_train, y_train)
    y_pred = RF.predict(X_test)
    r2[i] = r2_score(y_test, y_pred)
    mse[i] = mean_squared_error(y_test, y_pred)

print("Test R2: {:.4f} ± {:.4f}".format(np.mean(r2), np.std(r2)))
print("Test MSE: {:.4f} ± {:.4f}".format(np.mean(mse), np.std(mse)))

/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/honda/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/home/honda/anaconda3/lib/python3.6/site-package

Test R2: 0.7645 ± 0.0111
Test MSE: 0.9615 ± 0.0453


In [35]:
study = optuna.create_study()
study.optimize(objective_rf, n_trials=100)

[I 2019-03-28 11:48:58,732] Finished a trial resulted in value: 1.6698292243017374. Current best value is 1.6698292243017374 with parameters: {'max_depth': 5.6808253653308896, 'n_estimators': 27.408973177788376, 'max_features': 4, 'min_samples_split': 2, 'min_samples_leaf': 7}.
[I 2019-03-28 11:48:59,303] Finished a trial resulted in value: 1.192314717142425. Current best value is 1.192314717142425 with parameters: {'max_depth': 11.687340586483788, 'n_estimators': 31.086403951298298, 'max_features': 10, 'min_samples_split': 10, 'min_samples_leaf': 7}.
[I 2019-03-28 11:48:59,720] Finished a trial resulted in value: 1.787413267161701. Current best value is 1.192314717142425 with parameters: {'max_depth': 11.687340586483788, 'n_estimators': 31.086403951298298, 'max_features': 10, 'min_samples_split': 10, 'min_samples_leaf': 7}.
[I 2019-03-28 11:48:59,840] Finished a trial resulted in value: 1.5585647500765905. Current best value is 1.192314717142425 with parameters: {'max_depth': 11.68734

[I 2019-03-28 11:51:04,529] Finished a trial resulted in value: 1.203589037516287. Current best value is 0.9559830360833292 with parameters: {'max_depth': 81.44503944496788, 'n_estimators': 868.2335792329872, 'max_features': 5, 'min_samples_split': 7, 'min_samples_leaf': 1}.
[I 2019-03-28 11:51:05,142] Finished a trial resulted in value: 1.0751876260884976. Current best value is 0.9559830360833292 with parameters: {'max_depth': 81.44503944496788, 'n_estimators': 868.2335792329872, 'max_features': 5, 'min_samples_split': 7, 'min_samples_leaf': 1}.
[I 2019-03-28 11:51:11,359] Finished a trial resulted in value: 1.2821300974628533. Current best value is 0.9559830360833292 with parameters: {'max_depth': 81.44503944496788, 'n_estimators': 868.2335792329872, 'max_features': 5, 'min_samples_split': 7, 'min_samples_leaf': 1}.
[I 2019-03-28 11:51:11,823] Finished a trial resulted in value: 1.779600426774513. Current best value is 0.9559830360833292 with parameters: {'max_depth': 81.445039444967

[I 2019-03-28 11:52:18,833] Finished a trial resulted in value: 1.1042304144843025. Current best value is 0.9559830360833292 with parameters: {'max_depth': 81.44503944496788, 'n_estimators': 868.2335792329872, 'max_features': 5, 'min_samples_split': 7, 'min_samples_leaf': 1}.
[I 2019-03-28 11:52:35,845] Finished a trial resulted in value: 1.0649787530506916. Current best value is 0.9559830360833292 with parameters: {'max_depth': 81.44503944496788, 'n_estimators': 868.2335792329872, 'max_features': 5, 'min_samples_split': 7, 'min_samples_leaf': 1}.
[I 2019-03-28 11:52:38,433] Finished a trial resulted in value: 1.1335937195348178. Current best value is 0.9559830360833292 with parameters: {'max_depth': 81.44503944496788, 'n_estimators': 868.2335792329872, 'max_features': 5, 'min_samples_split': 7, 'min_samples_leaf': 1}.
[I 2019-03-28 11:52:41,422] Finished a trial resulted in value: 2.549202823210309. Current best value is 0.9559830360833292 with parameters: {'max_depth': 81.44503944496

[I 2019-03-28 11:53:26,596] Finished a trial resulted in value: 1.1215785196355599. Current best value is 0.9558994986586622 with parameters: {'max_depth': 35.322289568304406, 'n_estimators': 44.38473822847152, 'max_features': 8, 'min_samples_split': 6, 'min_samples_leaf': 3}.
[I 2019-03-28 11:53:32,364] Finished a trial resulted in value: 1.0901930687196015. Current best value is 0.9558994986586622 with parameters: {'max_depth': 35.322289568304406, 'n_estimators': 44.38473822847152, 'max_features': 8, 'min_samples_split': 6, 'min_samples_leaf': 3}.
[I 2019-03-28 11:53:47,850] Finished a trial resulted in value: 1.097794102847676. Current best value is 0.9558994986586622 with parameters: {'max_depth': 35.322289568304406, 'n_estimators': 44.38473822847152, 'max_features': 8, 'min_samples_split': 6, 'min_samples_leaf': 3}.
[I 2019-03-28 11:53:47,961] Finished a trial resulted in value: 1.8493076388446124. Current best value is 0.9558994986586622 with parameters: {'max_depth': 35.32228956

In [36]:
# Optimized

n = 10
r2 = np.zeros(n)
mse = np.zeros(n)

for i in range(n):
    RF = RandomForestRegressor(max_depth=35, n_estimators=44, max_features=8,
                              min_samples_split=6, min_samples_leaf=3)
    RF.fit(X_train, y_train)
    y_pred = RF.predict(X_test)
    r2[i] = r2_score(y_test, y_pred)
    mse[i] = mean_squared_error(y_test, y_pred)

print("Test R2: {:.4f} ± {:.4f}".format(np.mean(r2), np.std(r2)))
print("Test MSE: {:.4f} ± {:.4f}".format(np.mean(mse), np.std(mse)))

Test R2: 0.7724 ± 0.0053
Test MSE: 0.9292 ± 0.0218
